## Libraries

In [ ]:
!pip install torch_geometric
!pip install torch-spline-conv


In [2]:
import warnings
warnings.filterwarnings('ignore')

import os.path as osp
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV

## Loading Dataset

In [3]:

if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

dataset = 'Cora'
transform = T.Compose([
    T.RandomNodeSplit(num_val=500, num_test=500),
    T.TargetIndegree(),
])
path =  'data'
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]

Processing...
Done!


## Model

In [4]:


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset.num_features, 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, dataset.num_classes, dim=1, kernel_size=2)

    def forward(self):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

In [5]:


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    log_probs, accs = model(), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs




In [ ]:
for epoch in range(50):
    train()
    train_acc, test_acc = test()
    if epoch % 10 == 0:
         print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 000, Train: 0.4450, Test: 0.4180
Epoch: 001, Train: 0.4737, Test: 0.4520
Epoch: 002, Train: 0.5504, Test: 0.5080
Epoch: 003, Train: 0.6587, Test: 0.6100
Epoch: 004, Train: 0.7553, Test: 0.6980
Epoch: 005, Train: 0.8419, Test: 0.7840
Epoch: 006, Train: 0.8841, Test: 0.8400
Epoch: 007, Train: 0.8964, Test: 0.8540
Epoch: 008, Train: 0.9040, Test: 0.8660
Epoch: 009, Train: 0.9110, Test: 0.8700
Epoch: 010, Train: 0.9186, Test: 0.8700
Epoch: 011, Train: 0.9204, Test: 0.8720
Epoch: 012, Train: 0.9221, Test: 0.8800
Epoch: 013, Train: 0.9268, Test: 0.8780
Epoch: 014, Train: 0.9274, Test: 0.8860
Epoch: 015, Train: 0.9303, Test: 0.8880
Epoch: 016, Train: 0.9338, Test: 0.8880
Epoch: 017, Train: 0.9391, Test: 0.8900
Epoch: 018, Train: 0.9415, Test: 0.8940
Epoch: 019, Train: 0.9450, Test: 0.9000
Epoch: 020, Train: 0.9485, Test: 0.9040
Epoch: 021, Train: 0.9514, Test: 0.9040
Epoch: 022, Train: 0.9549, Test: 0.8940
Epoch: 023, Train: 0.9584, Test: 0.8960
Epoch: 024, Train: 0.9602, Test: 0.8940


In [ ]:
train_acc, test_acc = test()

In [ ]:
print(train_acc, test_acc)

0.9707259953161592 0.888


##  Measurement of avarage of accuracy, and time inference

In [7]:
import time
import statistics as stat


Eva_final=dict()


Base_model_Acc=[]
T_base_model=[]
T_biulding_model=[]

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
for i in range(5):


        model = Net().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

        start = time.time()
        for epoch in range(100):
            train()
            train_acc, test_acc = test()
            if epoch % 10 == 0:
                print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')
        end = time.time()
        t_building_model= end-start

        start = time.time()
        train_acc, test_acc = test()
        end = time.time()
        t_inference= end-start

        print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}, Time inference:{t_inference}")


        Base_model_Acc.append(test_acc)
        T_base_model.append(t_inference)
        T_biulding_model.append( t_building_model)

Epoch: 000, Train: 0.6721, Test: 0.5900
Epoch: 010, Train: 0.9069, Test: 0.8360
Epoch: 020, Train: 0.9514, Test: 0.8760
Epoch: 030, Train: 0.9690, Test: 0.8700
Epoch: 040, Train: 0.9696, Test: 0.8780
Epoch: 050, Train: 0.9742, Test: 0.8820
Epoch: 060, Train: 0.9772, Test: 0.8760
Epoch: 070, Train: 0.9742, Test: 0.8760
Epoch: 080, Train: 0.9760, Test: 0.8780
Epoch: 090, Train: 0.9801, Test: 0.8760
Train Accuracy: 0.9771662763466042, Test Accuracy: 0.868, Time inference:0.0033257007598876953
Epoch: 000, Train: 0.4719, Test: 0.4180
Epoch: 010, Train: 0.9280, Test: 0.8680
Epoch: 020, Train: 0.9537, Test: 0.8760
Epoch: 030, Train: 0.9707, Test: 0.8660
Epoch: 040, Train: 0.9719, Test: 0.8760
Epoch: 050, Train: 0.9731, Test: 0.8760
Epoch: 060, Train: 0.9713, Test: 0.8700
Epoch: 070, Train: 0.9813, Test: 0.8740
Epoch: 080, Train: 0.9766, Test: 0.8660
Epoch: 090, Train: 0.9766, Test: 0.8640
Train Accuracy: 0.977751756440281, Test Accuracy: 0.856, Time inference:0.0033025741577148438
Epoch: 000,

In [11]:
print(f'Base_model_Acc in ten time:')
print ("{:<10}|{:<10}|{:<10}|{:<15}|{:<10}"\
    .format(Base_model_Acc[0],Base_model_Acc[1],Base_model_Acc[2],\
        Base_model_Acc[3],Base_model_Acc[4]))
print ("-"*110)
print(f'Time Inference model in ten time:')
#print(T_base_model)
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}"\
    .format(T_base_model[0],T_base_model[1],T_base_model[2],\
        T_base_model[3],T_base_model[4]))

Base_model_Acc in ten time:
0.87      |0.868     |0.878     |0.864          |0.882     
--------------------------------------------------------------------------------------------------------------
Time Inference model in ten time:
0.0032656192779541016|0.0032324790954589844|0.0038950443267822266|0.0032536983489990234|0.0035254955291748047


In [12]:
import statistics as stat

base_model_accuracy_mean = stat.mean(Base_model_Acc)
base_model_accuracy_std =  stat.stdev(Base_model_Acc)
desc_auc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)
print(f"Auc: {desc_auc}"  )

Eva_final.update({'base model accuracy':float(format(base_model_accuracy_mean, '.4f'))})

t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)
desc_T = "{:.3f} ± {:.3f}".format(t_base_model_mean,t_base_model_std)

print(f"Time inference :{desc_T}")
Eva_final.update({'time inference of base model':float(format(t_base_model_mean, '.6f'))})

Auc: 0.872 ± 0.008
Time inference :0.004 ± 0.000


In [16]:
def get_num_parameters(model, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


def get_model_size(model, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width


print(f"Model Size: {get_model_size(model)}")

In [18]:
print(f"Model Size: {get_model_size(model)}")

Model Size: 2212576
